# Brazilian Houses To Rent: XGBoost

Hi everyone! This is my first kernel, hope I didn't make mistakes. Be kind :)

In [ ]:
# import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import mean

color = sns.color_palette()
sns.set_style('darkgrid')


from scipy import stats
from scipy.stats import norm, skew

data = pd.read_csv('../input/brasilian-houses-to-rent/houses_to_rent_v2.csv')
data.head()


Let's rename some columns to remove the currency info.

In [ ]:
data = data.rename(columns={"hoa (R$)":"hoa", "rent amount (R$)":"rent amount",
                            "property tax (R$)": "property tax", "fire insurance (R$)":"fire insurance",
                            "total (R$)":"total price"})


data['floor'].mask(data['floor'] == '-', '0', inplace=True)
data['floor'] = data['floor'].astype('int')
data.head()


Let's check if there are missing values.


In [ ]:
print(data.isnull().sum())

Zero missing values! Now let's search for outliers

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(15,5))
sns.boxplot(x=data['area'], ax = axs[0])
sns.boxplot(x=data['rent amount'], ax = axs[1])
sns.boxplot(x=data['floor'], ax = axs[2])
plt.show()

There are a lot of outliers, but some of them are really far from the third quartile! They could surely be a problem for a model. Let's remove them

In [ ]:
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3- Q1
print(IQR, data.shape)

In [ ]:
data = data[~((data < (Q1 - 5*IQR)) | (data > (Q3 + 5*IQR))).any(axis=1)]
data.shape

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(10,5))
sns.boxplot(x=data['area'], ax = axs[0])
sns.boxplot(x=data['rent amount'], ax = axs[1])
sns.boxplot(x=data['floor'], ax = axs[2])
plt.show()

# Correlation matrix


In [ ]:
plt.figure(figsize=(20,10))
c = data.corr()
sns.heatmap(c,cmap='Blues', annot=True)

There is a strong correlation between: 
- fire insurance and rent amount
- total price and rent amount
- total price and fire insurance

# A bunch of descriptive analysis
1. Average rent amount by city
2. Average rent amount by city and by animal acceptance
3. Average rent amount by n. of parking spaces




In [ ]:
from numpy import mean
plt.figure(figsize = (8,4))
sns.barplot(data = data, x='city', y='rent amount', estimator=mean, palette = "Blues")
plt.ylabel('Avg Rent Amount')
plt.xlabel('')
plt.title('Average Rent Amount by City')
plt.legend('')

plt.figure(figsize = (8,4))
sns.barplot(x='city', y='rent amount', data = data, hue = 'animal', palette='Blues', estimator = mean)
plt.ylabel('Avg Rent Amount')
plt.title('Average Rent Amount by City and by Animal Acceptance')

plt.figure(figsize = (8,4)) 
sns.barplot(x='parking spaces', y='rent amount', data = data, palette='Blues', estimator=mean)
plt.ylabel('Avg Rent Amount')
plt.title('Averace Rent Amount by n. of Parking Spaces')
plt.show()



Let's check every column's type. 
- We changed floor type to integer(the ground floor is referred to as '-' so we change it to 0).
- We change city and animal type to category so that we can convert them into dummy variables.
- We convert city's names into numeric categories

In [ ]:
data["city"] = data["city"].astype('category')
data["animal"] = data["animal"].astype('category')
cleanup = {"city": {"São Paulo" : 1, "Porto Alegre" : 2, "Rio de Janeiro": 3, "Campinas":4, "Belo Horizonte":5},
          "animal": {"not acept":0, "acept":1},
          "furniture": {"not furnished":0, "furnished":1}}
data.replace(cleanup, inplace=True)
data.head()

We convert 'city' into dummy variables.

In [ ]:
data = pd.get_dummies(data, columns=['city'])
data = data.drop(columns=['city_1'])

In [ ]:
data = data[['city_2', 'city_3', 'city_4', 'city_5', 'area', 'rooms','bathroom', 'parking spaces', 'floor', 'furniture', 'hoa', 'property tax', 'fire insurance', 'total price', 'animal', 'rent amount']]
data.head()

In [ ]:
col = ['city_2', 'city_3', 'city_4', 'city_5', 'area', 'rooms','bathroom', 'parking spaces', 'floor', 'furniture', 'hoa', 'property tax', 'total price', 'animal']
sns.pairplot(data[col], size=2)
plt.show

In [ ]:
data['RentAmountLog'] = np.log(data['rent amount'])
data = data.drop(columns=['total price', 'fire insurance'])

from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)

In [ ]:
corr = train.corr().abs()
corr.RentAmountLog[corr.RentAmountLog >= 0.5].sort_values(ascending=False)

In [ ]:
plt.subplot2grid((2,1), (0,0))
train['rent amount'].plot(kind='kde')
plt.title('Rent Amount')

plt.subplot2grid((2,1),(1,0))
train['RentAmountLog'].plot(kind='kde')
plt.title('Rent Amount Log')

plt.subplots_adjust(hspace=0.5, wspace = 0.3)
plt.show()




In [ ]:
y_train = train['RentAmountLog']
y_test = test['RentAmountLog']
x_train = train.drop(columns=['RentAmountLog', 'rent amount'])
x_test = test.drop(columns=['RentAmountLog', 'rent amount'])

In [ ]:
import xgboost
from sklearn.metrics import mean_squared_error

xgb = xgboost.XGBRegressor(min_child_weight=1.8, silent=1, colsample_bytree=0.8, subsample=1,
                           learning_rate=0.01, max_depth=4, n_estimators=3000,
                           reg_lambda=0.1, gamma = 0, silence=False, reg_alpha=0.1, nthr = -1 )

xgb.fit(x_train, y_train)
y_test_pred = xgb.predict(x_test)


In [ ]:
RMSE = np.sqrt(mean_squared_error(y_test_pred, y_test))
print(RMSE.round(6))